In [ ]:
CREATE OR REPLACE WAREHOUSE snowpark_opt_wh WITH
  WAREHOUSE_SIZE = 'MEDIUM'
  WAREHOUSE_TYPE = 'SNOWPARK-OPTIMIZED';

In [ ]:
CREATE WAREHOUSE so_warehouse WITH
  WAREHOUSE_SIZE = 'LARGE'
  WAREHOUSE_TYPE = 'SNOWPARK-OPTIMIZED'
  RESOURCE_CONSTRAINT = 'MEMORY_16X_X86';

In [ ]:
USE ROLE ACCOUNTADMIN;


CREATE OR REPLACE DATABASE DASH_DB;
CREATE OR REPLACE SCHEMA DASH_SCHEMA;

USE DASH_DB.DASH_SCHEMA;
USE WAREHOUSE DASH_S;

In [ ]:
create or replace table support_ticket_category (
  category string
);

INSERT INTO support_ticket_category (category) VALUES 
  ('Roaming fees'), 
  ('Slow data speed'), 
  ('Lost phone'), 
  ('Add new line'), 
  ('Closing account');


In [ ]:
USE DASH_DB.DASH_SCHEMA;
USE WAREHOUSE DASH_S;

In [ ]:
create or replace table support_tickets as (
    SELECT 
      category, 
      TRY_PARSE_JSON(
        SNOWFLAKE.CORTEX.COMPLETE(
          'llama3.1-405b',
          CONCAT(
            'Please provide 25 examples of customer service calls in a telecom company for the following category:', category, '. Provide detailed and realistic scenarios that customer service representatives might encounter. Ensure the examples are diverse and cover various situations within each category. Please put the  examples into a JSON list. Each element in JSON list should include the following: {"scenario": <scenario>, "request": <detailed request from the customer, which usually is less than 3 sentences.>}. Only include JSON in output and no other words.'))) as tickets
    from support_ticket_category
);

In [ ]:
USE WAREHOUSE SO_WAREHOUSE;

create or replace table flatten_support_tickets as (
select 
    category, 
    abs(hash(value:request)) % 10000000 as id,
    value:request as request, 
    value:scenario as scenario
from support_tickets, lateral flatten(input => tickets) 
);

In [ ]:
select * 
from DASH_DB.DASH_SCHEMA.SUPPORT_TICKETS, lateral flatten(input => tickets) 


In [ ]:
create or replace table rate_support_tickets as (
    SELECT category, id, request, scenario, TRY_PARSE_JSON(SNOWFLAKE.CORTEX.COMPLETE('llama3.1-405b', CONCAT('You are a judge to verify if a the support ticket received in a telecom company is realistic, and valid, please give scores from 1 to 5 for each category and give your final recommendation for the given question. Support Ticket: ', request, ' Please give the score in JSON format alone following this example: "{"realistic": 5, "valid": 4}".  You can put a reason into the result JSON as "reason": <reason>. Only include JSON in the output and no other words.'))) as rating
    from flatten_support_tickets
);

In [ ]:
create or replace table filtered_support_tickets as (
    select * from rate_support_tickets where rating['realistic'] >= 4 and rating['valid'] >= 4
);

In [ ]:
CREATE OR REPLACE FUNCTION CATEGORIZE_PROMPT_TEMPLATE(request STRING)
RETURNS STRING
LANGUAGE SQL
AS
$$
CONCAT('You are an agent that helps organize requests that come to our support team. 

The request category is the reason why the customer reached out. These are the possible types of request categories:

Roaming fees
Slow data speed
Lost phone
Add new line
Closing account

Try doing it for this request and return only the request category only.

request: ', request)
$$
;

In [ ]:
SELECT id, SNOWFLAKE.CORTEX.COMPLETE('llama3-8b', CATEGORIZE_PROMPT_TEMPLATE(request)) FROM filtered_support_tickets;


In [ ]:
create or replace table training_data as (
    SELECT * from filtered_support_tickets where ID % 10 < 7 
);

create or replace table validation_data as (
    SELECT * from filtered_support_tickets where ID % 10 >=  
);

In [ ]:
create or replace table  model_accuracy_log (date_timestamp TIMESTAMP, llm_model string, accuracy float)

In [ ]:
SET model_name = 'llama3.1-405b';
USE DATABASE DASH_DB;
USE SCHEMA DASH_SCHEMA;

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE accuracy_flag as (
with predictions as (
SELECT id, request,category as actual_category, TRIM(SNOWFLAKE.CORTEX.COMPLETE($model_name, CATEGORIZE_PROMPT_TEMPLATE(request))) as model_category
FROM filtered_support_tickets), ACCURACY AS (SELECT CASE WHEN POSITION(actual_category, model_category) > 0 THEN 1 ELSE 0 END AS true_positives
from predictions)
select * from ACCURACY)

In [ ]:
set accuracy = (select SUM(TRUE_POSITIVES)/COUNT(*) AS ACCURACY FROM DASH_DB.DASH_SCHEMA.ACCURACY_FLAG)

In [ ]:
insert into model_accuracy_log values (CURRENT_TIMESTAMP(2), $model_name, $accuracy)

In [ ]:
select * EXCLUDE RN FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY LLM_MODEL,ACCURACY ORDER BY DATE_TIMESTAMP) AS RN from model_accuracy_log)
WHERE RN=1 ORDER BY DATE_TIMESTAMP DESC 